In [1]:
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

/var/folders/xz/1q44kbwx25g7j6b7qsjs_x4r0000gp/T/ipykernel_24791/1551533013.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_path = '../../data/clasdb_pi_plus_n.txt'

In [3]:
df = pd.read_csv(data_path, delimiter='\t', header=None)
df.columns = ['Ebeam', 'W', 'Q2', 'cos_theta', 'phi', 'dsigma_dOmega', 'error', 'id']
df.loc[8314:65671, 'Ebeam'] = 5.754  # peculiarity of this dataset.
df = df[~((df.Ebeam == 5.754) & (~df.Q2.isin([1.715, 2.050, 2.445, 2.915, 3.480, 4.155])))] # peculiarity of this dataset #2
df['phi'] = df.phi.apply(lambda x: math.radians(x))
df['weight'] = df['error'].apply(lambda x: x and 1 / x or 100)  # x and 1 / x or 100  is just a reversed error but with validation 1/0 error in this case it will return 100
df = df.drop('id', axis=1)
df = df.drop_duplicates(subset=['Ebeam', 'W', 'Q2', 'cos_theta', 'phi'])
df['cos_theta_rounded'] = df['cos_theta'].apply(lambda x: round(x,2))

In [4]:
df_a = pd.read_csv('../../data/final_table.csv', header=None, sep='\t',
                    names=['Channel', 'MID', 'Wmin', 'Wmax', 'Q2min', 'Q2max', 'Cos(theta)', 'sigma_t', 'd_sigma_t',
                    'sigma_l', 'd_sigma_l', 'sigma_tt', 'd_sigma_tt', 'sigma_lt', 'd_sigma_lt', 'eps'])
df_a = df_a[df_a.Channel.isin([8, 14, 41, 141])]
df_a['W'] = (df_a['Wmax'] + df_a['Wmin'])/2
df_a['Q2'] = (df_a['Q2max'] + df_a['Q2min'])/2
df_a['cos_theta'] = df_a['Cos(theta)']
df_a['cos_theta_rounded'] = df_a['cos_theta'].apply(lambda x: round(x,2))

,Ebeam,W,Q2,cos_theta,phi,dsigma_dOmega,error,weight,cos_theta_rounded
0,1.515,1.11,0.3,0.991445,0.261799,15.3700,5.264366,0.189956,0.99
1,1.515,1.11,0.3,0.991445,0.785398,4.5110,1.743136,0.573679,0.99
2,1.515,1.11,0.3,0.991445,1.308997,4.4780,1.611260,0.620632,0.99
3,1.515,1.11,0.3,0.991445,1.832596,5.1360,1.523529,0.656371,0.99
4,1.515,1.11,0.3,0.991445,2.356194,5.0780,1.219442,0.820047,0.99
...,...,...,...,...,...,...,...,...,...
98022,5.499,2.01,4.0,0.975000,3.730641,0.1012,0.043165,23.166714,0.97
98023,5.499,2.01,4.0,0.975000,3.992441,0.1199,0.076638,13.048361,0.97
98024,5.499,2.01,4.0,0.975000,4.646939,0.1578,0.095391,10.483159,0.97
98025,5.499,2.01,4.0,0.975000,4.777839,0.2346,0.158557,6.306878,0.97


In [19]:
df_data = df.merge(df_a, how='left', on=['W', 'Q2', 'cos_theta_rounded'])

df_data['A'] = df_data.sigma_t + df_data.sigma_l
df_data['A_error'] = df_data.d_sigma_t + df_data.d_sigma_l

df_data['B'] = df_data.sigma_tt
df_data['B_error'] = df_data.d_sigma_tt

df_data['C'] = df_data.sigma_lt
df_data['C_error'] = df_data.d_sigma_lt


df_data['cos_theta'] = df_data['cos_theta_x']
df_data = df_data[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi',
                    'dsigma_dOmega', 'error', 'A', 'A_error',
                    'B', 'B_error', 'C', 'C_error', 'eps']]

In [55]:
# df_data.to_csv('../../data/df_with_ABC.csv')